## **Function Calling LLMs - Team Project**

In [2]:
import openai
import json
import os

import pandas as pd
from pprint import pprint
from dotenv import load_dotenv


load_dotenv()
openai.api_key = os.environ.get("API_KEY")

## Data

##### Dataset 1: Most Popular Albums on Spotify

Source: https://www.kaggle.com/datasets/tobennao/rym-top-5000/

In [21]:
album_columns_to_keep = ["release_name",    # Name of the album 
                            "artist_name",     # Name of the artist/band/group
                            "release_date",    # Date the album was released
                            "primary_genres",  # Primary genre classifications
                            "secondary_genres",# Secondary genre classifications
                            "descriptors",     # Album tags
                            "avg_rating",      # Average rating, on a scale of 0-5
                            "rating_count",    # The number of ratings
                            "review_count"     # The number of reviews
                        ]

albums_popular = pd.read_csv("./data/popular_albums.csv", usecols=album_columns_to_keep)


# one attribute for release_date
def combine_and_deduplicate_genres(primary, secondary):
    combined_genres = set()

    if primary and not isinstance(primary, float):
        combined_genres.update(primary.split(', '))

    if secondary and not isinstance(secondary, float):
        combined_genres.update(secondary.split(', '))

    return ', '.join(combined_genres)


albums_popular['genres'] = albums_popular.apply(lambda row: combine_and_deduplicate_genres(
    row['primary_genres'], row['secondary_genres']), axis=1)

albums_popular.drop(['primary_genres', 'secondary_genres'], axis=1, inplace=True)


# reorder:
new_order = ["release_name", "artist_name", "release_date", "genres",
             "descriptors", "avg_rating", "rating_count", "review_count"]
albums_popular = albums_popular[new_order]

In [22]:
album_columns_to_keep = ["release_name",    # Name of the album
                         "artist_name",     # Name of the artist/band/group
                         "release_date",    # Date the album was released
                         "genres",          # Genre
                         "descriptors",     # Album tags
                         "avg_rating",      # Average rating, on a scale of 0-5
                         "rating_count",    # The number of ratings
                         "review_count"     # The number of reviews
                         ]

albums_long_tail = pd.read_csv("./data/albums_long_tail.csv", usecols=album_columns_to_keep)

In [38]:
ALBUMS = pd.concat([albums_popular[:500], albums_long_tail], ignore_index=True)

print(f"Attributes: {ALBUMS.columns.to_list()}")
print(f"Number of attributes (columns): {ALBUMS.shape[1]}")
print(f"Number of examples (rows): {ALBUMS.shape[0]}")

ALBUMS[-10:].head(n=10)

Attributes: ['release_name', 'artist_name', 'release_date', 'genres', 'descriptors', 'avg_rating', 'rating_count', 'review_count']
Number of attributes (columns): 8
Number of examples (rows): 619


,release_name,artist_name,release_date,genres,descriptors,avg_rating,rating_count,review_count
609,Dismantled Into Juice,Blawan,2023-05-17,"UK Bass, Wonky Techno, Deconstructed Club, Wonky","mechanical, rhythmic, hypnotic, dissonant, raw...",3.34,527,2.0
610,But Here We Are,Foo Fighters,2023-06-02,"Alternative Rock, Power Pop, Post-Grunge, Shoe...","death, male vocalist, melodic, anthemic, bitte...",3.56,5118,76.0
611,Metro Boomin Presents Spider-Man: Across the S...,Metro Boomin,2023-06-02,"Film Soundtrack, Pop Rap, Trap, Afrobeats, Con...","male vocalist, female vocalist, optimistic, tr...",2.96,2295,19.0
612,Formal Growth in the Desert,Protomartyr,2023-06-02,"Post-Punk, Art Punk, Gothic Rock, Noise Rock","male vocalist, dark, rhythmic, atmospheric, de...",3.45,2785,29.0
613,Bunny,Beach Fossils,2023-06-02,"Jangle Pop, Indie Pop, Dream Pop, Indie Surf, ...","male vocalist, mellow, calm, soft, ethereal, w...",3.29,929,9.0
614,Everyone's Crushed,Water From Your Eyes,2023-05-26,"Experimental Rock, Art Pop, Neo-Psychedelia, P...","apathetic, urban, dissonant, noisy, energetic,...",3.32,1149,15.0
615,Aperture,Hannah Jadagu,2023-05-19,"Indie Pop, Dream Pop, Bedroom Pop","bittersweet, melancholic, sentimental, energet...",3.44,293,4.0
616,More Photographs (A Continuum),Kevin Morby,2023-05-26,"Folk Rock, Singer-Songwriter, Indie Folk, Amer...","lonely, sentimental, melodic, male vocalist, w...",3.13,143,2.0
617,Perfume,NCT DOJAEJUNG,2023-04-17,"Contemporary R&B, K-Pop, Dance-Pop, Future Bas...","sensual, male vocalist, melodic, rhythmic, rom...",3.44,360,3.0
618,AESTHETIC,tripleS / +(KR)ystal Eyes,2023-05-04,"K-Pop, Dance-Pop, Contemporary R&B, New Jack S...","female vocalist, warm, rhythmic, lush, melodic...",3.50,752,5.0


##### Dataset 2: Most Streamed Tracks on Spotify

Source: https://www.kaggle.com/datasets/nelgiriyewithana/top-spotify-songs-2023

In [25]:
song_columns_to_keep = ['track_name',           # Name of the song
                        'artist(s)_name',       # Name of the artist(s) of the song
                        'artist_count',         # Number of artists contributing to the song
                        'released_year',        # Year when the song was released
                        'released_month',       # Month when the song was released
                        'released_day',         # Day of the month when the song was released
                        'streams',              # Total number of streams on Spotify
                        'bpm',                  # Beats per minute, a measure of song tempo
                        'key',                  # Key of the song
                        'mode',                 # Mode of the song (major or minor)
                        'danceability_%',       # Percentage indicating how suitable the song is for dancing
                        'valence_%',            # Positivity of the song's musical content
                        'energy_%',             # Perceived energy level of the song
                        'acousticness_%',       # Amount of acoustic sound in the song
                        'instrumentalness_%',   # Amount of instrumental content in the song
                        'liveness_%',           # Presence of live performance elements
                        'speechiness_%'         # Amount of spoken words in the song
                        ]


songs_short_tail = pd.read_csv("./data/tracks.csv", encoding_errors="ignore")

# one attribute for release_date
songs_short_tail['release_date'] = pd.to_datetime(songs_short_tail['released_year'].astype(str) + '-' +
                                       songs_short_tail['released_month'].astype(str) + '-' +
                                       songs_short_tail['released_day'].astype(str))

songs_short_tail.drop(['released_year', 'released_month',
        'released_day'], axis=1, inplace=True)


# reorder:
new_order = ['track_name', 'artist(s)_name', 'artist_count', 'release_date', 'streams', 'bpm', 'key', 'mode', 
             'danceability_%', 'valence_%', 'energy_%', 'acousticness_%', 'instrumentalness_%', 'liveness_%', 'speechiness_%']
songs_short_tail = songs_short_tail[new_order]

In [26]:
songs_long_tail = pd.read_csv("./data/tracks_long_tail.csv", encoding_errors="ignore")

In [37]:
SONGS = pd.concat([songs_short_tail[:500], songs_long_tail], ignore_index=True)

print(f"Attributes: {SONGS.columns.to_list()}")
print(f"Number of attributes (columns): {SONGS.shape[1]}")
print(f"Number of examples (rows): {SONGS.shape[0]}")

SONGS[495:].head(n=10)

Attributes: ['track_name', 'artist(s)_name', 'artist_count', 'release_date', 'streams', 'bpm', 'key', 'mode', 'danceability_%', 'valence_%', 'energy_%', 'acousticness_%', 'instrumentalness_%', 'liveness_%', 'speechiness_%', 'album', 'explicit', 'popularity', 'duration_in_min']
Number of attributes (columns): 19
Number of examples (rows): 960


,track_name,artist(s)_name,artist_count,release_date,streams,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%,album,explicit,popularity,duration_in_min
495,Run Rudolph Run - Single Version,Chuck Berry,1,1958-01-01 00:00:00,245350949,152,G,Minor,69,94,71,79,0,7,8,NaN,NaN,NaN,NaN
496,Jingle Bells - Remastered 1999,Frank Sinatra,1,1957-01-01 00:00:00,178660459,175,G#,Major,51,94,34,73,0,10,5,NaN,NaN,NaN,NaN
497,Far,SZA,1,2022-12-09 00:00:00,51641685,116,D,Major,61,48,55,67,0,16,8,NaN,NaN,NaN,NaN
498,On Time (with John Legend),"John Legend, Metro Boomin",2,2022-12-02 00:00:00,78139948,80,F,Minor,33,51,59,76,0,44,6,NaN,NaN,NaN,NaN
499,GAT��,"Maldy, Karol G",2,2022-08-25 00:00:00,322336177,93,B,Minor,63,34,86,26,0,21,39,NaN,NaN,NaN,NaN
500,3D (feat. Jack Harlow),"Jung Kook, Jack Harlow",2,2023-11-03,NaN,108,C#,Major,86,89,83,4,0,9,11,GOLDEN,True,85.0,3.363533
501,Closer to You (feat. Major Lazer),"Jung Kook, Major Lazer",2,2023-11-03,NaN,113,D,Minor,79,50,66,12,1,11,5,GOLDEN,False,86.0,2.849917
502,Seven (feat. Latto) (Explicit Ver.),"Jung Kook, Latto",2,2023-11-03,NaN,124,B,Major,79,88,84,32,0,8,5,GOLDEN,True,87.0,3.059183
503,Standing Next to You,Jung Kook,1,2023-11-03,NaN,106,D,Minor,72,82,81,5,0,34,10,GOLDEN,False,96.0,3.433667
504,Yes or No,Jung Kook,1,2023-11-03,NaN,83,C#,Major,68,89,84,18,0,8,9,GOLDEN,False,88.0,2.459283


## Functions

#### Albums

In [46]:
def top_rated_albums(n=10):
    """
    Returns the top-rated albums based on average rating.
    
    Parameters:
        n (int): The number of albums to return. Default is 10.
    
    Returns:
        list[dict]: A list of dictionaries representing the top-rated albums.
    """
    top_rated = ALBUMS.sort_values(by='avg_rating', ascending=False).head(n)
    return top_rated.to_dict(orient='records')

def most_reviewed_albums(n=10):
    """
    Returns the most reviewed albums.
    
    Parameters:
        n (int): The number of albums to return. Default is 10.
    
    Returns:
        list[dict]: A list of dictionaries representing the most reviewed albums.
    """
    most_reviewed = ALBUMS.sort_values(by='review_count', ascending=False).head(n)
    return most_reviewed.to_dict(orient='records')
    
def albums_by_artist(artist_name):
    """
    Returns all albums by a given artist.
    
    Parameters:
        artist_name (str): The name of the artist.
    
    Returns:
        list[dict]: A list of dictionaries representing the albums by the given artist.
    """
    albums = ALBUMS[ALBUMS['artist_name'] == artist_name]
    return albums.to_dict(orient='records')

def artist_by_album(album_name):
    """
    Returns the artist of an album by its name.

    Parameters:
        album_name (str): The name of the album.

    Returns:
        dict: A dictionary with the album name and the artist.
              If the album is not found, the dictionary will be empty.
    """
    
    artist_dict = {}
    
    # Search for the album by name and populate the dictionary
    album = ALBUMS[ALBUMS['release_name'] == album_name]
    if not album.empty:
        artist_dict['Album name'] = album_name
        artist_dict['artist'] = album.iloc[0]['artist_name']
    
    return artist_dict


#### Songs

In [47]:
def top_streamed_songs(n=10):
    """
    Returns the top-streamed songs.
    
    Parameters:
        n (int): The number of songs to return. Default is 10.
    
    Returns:
        list[dict]: A list of dictionaries representing the top-streamed songs.
    """
    top_songs = SONGS.sort_values(by='streams', ascending=False).head(n)
    return top_songs.to_dict(orient='records')

def songs_in_spotify_playlists(n=10):
    """
    Returns the top songs featured in the most Spotify playlists.
    
    Parameters:
        n (int): The number of songs to return. Default is 10.
    
    Returns:
        list[dict]: A list of dictionaries representing the songs featured in the most Spotify playlists.
    """
    top_playlist_songs = SONGS.sort_values(by='in_spotify_playlists', ascending=False).head(n)
    return top_playlist_songs.to_dict(orient='records')

def songs_by_artist(artist_name):
    """
    Returns all songs by a given artist.
    
    Parameters:
        artist_name (str): The name of the artist.
    
    Returns:
        list[dict]: A list of dictionaries representing the songs by the given artist.
    """
    songs = SONGS[SONGS['artist(s)_name'] == artist_name]
    return songs.to_dict(orient='records')

def artist_by_song(song_name):
    """
    Returns the artist of the song.

    Parameters:
        song_name (str): The name of the song.

    Returns:
        dict: A dictionary with the song name and the artist.
              If the song is not found, the dictionary will be empty.
    """
    
    artist_dict = {}
    
    # Search for the song by name and populate the dictionary
    song = SONGS[SONGS['track_name'] == song_name]
    if not song.empty:
        artist_dict['Song'] = song_name
        artist_dict['Artist'] = song.iloc[0]['artist(s)_name']
    
    return artist_dict

##### Metadata

In [48]:
def filter_functions(functions_list, function_metadata):
    function_names = [func.__name__ for func in functions_list]
    filtered_metadata = [meta for meta in function_metadata if meta.get('name') in function_names]
    return functions_list, filtered_metadata

def describe_function(available_functions):
    return [meta["description"] for meta in available_functions[1]]

## LLM

In [49]:
from enum import Enum
from openai.openai_object import OpenAIObject

class Role(Enum):
    ASSISTANT = "assistant"
    FUNCTION = "function"
    SYSTEM = "system"
    USER = "user"
    
class Model(Enum):
    GPT3 = "gpt-3.5-turbo-0613"
    GPT4 = "gpt-4-0613"

class FunctionNotFoundError(Exception):
    def __init__(self, function_name, function_args):
        self.function_name = function_name
        self.function_args = function_args
        super().__init__(f"Error finding function {function_name} with arguments {function_args}")

class FunctionExecutionError(Exception):
    def __init__(self, function_name, function_args):
        self.function_name = function_name
        self.function_args = function_args
        super().__init__(f"Error executing function {function_name} with arguments {function_args}")


class Response:
    def __init__(self, message:dict):
        self._message = message
        
    @classmethod
    def from_api(cls, openai_response:OpenAIObject):
        _message = openai_response["choices"][0]["message"]
        return cls(_message)
    
    @property
    def message(self) -> str:
        return self._message["content"]

    @property
    def role(self) -> str:
        return self._message["role"]

    @property
    def function(self) -> dict:
        return self._message.get("function_call")

    @property
    def is_function_call(self) -> bool:
        return self.function is not None
    
    def to_dict(self) -> dict:
        return {**self._message}
    
    def __str__(self):
        return f"Response({self.role}: {self.message}, with function:{self.function})"


class Conversation:
    def __init__(self):
        self._messages = []

    @property
    def messages_as_dicts(self):
        return [message.to_dict() for message in self._messages]

    def send(self, model, functions) -> Response:

        args = {"model":model, 
                "messages":self.messages_as_dicts}
        
        if len(functions) > 0:
            args.update({"function_call":"auto",
                         "functions":functions})
            
        response = openai.ChatCompletion.create(**args)
        return Response.from_api(response)

    def add(self, message_or_response):
        if isinstance(message_or_response, Response):
            message = message_or_response
        else:
            message = Response(message_or_response)

        self._messages.append(message)        
        return self
    
    def __str__(self):
        return f"{self.messages_as_dicts}"
    

def handle_function(function:dict, functions) -> json:
    """Invoke function and return result"""
    function_name, function_args = function["name"], json.loads(function["arguments"])
    
    def get_function_by_name():
        for func in functions:
            if func.__name__ == function_name:
                return func
        return None
    
    function_to_call = get_function_by_name()
    if function_to_call is not None:
        try:
            result = json.dumps(function_to_call(**function_args))
            return result
        except:
            raise FunctionExecutionError(function_name, function_args)
    else:
        raise FunctionNotFoundError(function_name, function_args)


# TODO
def handle_error(error, retry):
    raise error


max_iterations = 5
def chat(conversation:Conversation, model:str, functions:tuple) -> str:
    iteration = 0
    while iteration < max_iterations:
        iteration +=1
        response = conversation.send(model, functions[1])
        conversation.add(response)
        
        if response.is_function_call:
            try:
                result = handle_function(response.function, functions[0])
                conversation.add({"role": Role.FUNCTION.value, "content": result, "name":response.function["name"]})
            except (FunctionNotFoundError, FunctionExecutionError) as error:
                handle_error(error=error, retry=False)
        else:
            return response.message
        

def handle_function_on_server(function):
    function_name, function_args = function["name"], json.loads(
        function["arguments"])
    
    import requests
    
    URL = f"http://localhost:5000/function_call/{function_name}?"
    
    response = requests.get(URL, params=function_args)
    if response.ok:
        data = response.json()
        return data["result"]
    else:
        raise FunctionExecutionError # TODO
    
    

PORT = 5000
def chat_web(conversation: Conversation, model: str, function_metadata: list) -> str:
    iteration = 0
    while iteration < max_iterations:
        iteration += 1
        response = conversation.send(model, function_metadata)
        conversation.add(response)

        if response.is_function_call:
            try:
                result = handle_function_on_server(response.function)
                conversation.add(
                    {"role": Role.FUNCTION.value, "content": result, "name": response.function["name"]})
            except (FunctionNotFoundError, FunctionExecutionError) as error:
                handle_error(error=error, retry=False)
        else:
            return response.message

## Benchmark

In [50]:
all_functions = [top_rated_albums, most_reviewed_albums, albums_by_artist, top_streamed_songs, songs_in_spotify_playlists, songs_by_artist, artist_by_song, artist_by_album]

with open('functions.json', 'r') as file:
    function_metadata = json.load(file)

In [51]:
print("Functions: " + str(len(all_functions)))
print(all_functions)

print()
print("Metadata: " + str(len(function_metadata)))
for meta_data in function_metadata:
    print(f"{meta_data['name']}: {meta_data['description']}")

Functions: 8
[<function top_rated_albums at 0x000002142D4D9940>, <function most_reviewed_albums at 0x000002142D4D8C20>, <function albums_by_artist at 0x000002142D4D8D60>, <function top_streamed_songs at 0x000002142D4DA2A0>, <function songs_in_spotify_playlists at 0x000002142D4DA660>, <function songs_by_artist at 0x000002142D4D9080>, <function artist_by_song at 0x000002142D4D98A0>, <function artist_by_album at 0x000002142D4D9120>]

Metadata: 8
top_rated_albums: Retrieves the records that have been valued uppermost.
most_reviewed_albums: Fetches the records that have been reviewed most frequently.
albums_by_artist: Gets albums of specified singer.
top_streamed_songs: Retrieves the trendy tracks which the vast majority has been influenced by.
songs_in_spotify_playlists: Retrieves the top-added musical pieces in Spotify playlists.
songs_by_artist: Fetches the music recording that has been released by the singer.
artist_by_song: Fetches the singer who released a stated track.
artist_by_albu

#### Functions

In [52]:
available_functions = filter_functions(all_functions, function_metadata)

conversation = Conversation()
conversation.add({"role": Role.SYSTEM.value, "content": "Answer briefly."}) \
            .add({"role": Role.USER.value, "content": "Could you please retrieve all songs of Kendrick Lamar?"})

result = chat(conversation, model=Model.GPT3.value, functions=available_functions)
print(result)

Here are some songs by Kendrick Lamar:

1. Track: HUMBLE.
   Artist: Kendrick Lamar
   Released: March 30, 2017
   Streams: 1,929,770,265

2. Track: N95
   Artist: Kendrick Lamar
   Released: May 13, 2022
   Streams: 301,242,089

3. Track: United In Grief
   Artist: Kendrick Lamar
   Released: May 13, 2022
   Streams: 156,898,322

4. Track: Rich Spirit
   Artist: Kendrick Lamar
   Released: May 13, 2022
   Streams: 173,702,135

5. Track: Count Me Out
   Artist: Kendrick Lamar
   Released: May 13, 2022
   Streams: 126,191,104

6. Track: Worldwide Steppers
   Artist: Kendrick Lamar
   Released: May 13, 2022
   Streams: 61,739,839

7. Track: Rich - Interlude
   Artist: Kendrick Lamar
   Released: May 13, 2022
   Streams: 41,210,087

8. Track: Crown
   Artist: Kendrick Lamar
   Released: May 13, 2022
   Streams: 42,485,571

9. Track: Auntie Diaries
   Artist: Kendrick Lamar
   Released: May 13, 2022
   Streams: 37,778,188

10. Track: Mirror
    Artist: Kendrick Lamar
    Released: May 13, 

In [53]:
pprint(conversation.messages_as_dicts)

[{'content': 'Answer briefly.', 'role': 'system'},
 {'content': 'Could you please retrieve all songs of Kendrick Lamar?',
  'role': 'user'},
 {'content': None,
  'function_call': <OpenAIObject at 0x2142d420f50> JSON: {
  "name": "songs_by_artist",
  "arguments": "{\n  \"artist_name\": \"Kendrick Lamar\"\n}"
},
  'role': 'assistant'},
 {'content': '[{"track_name": "HUMBLE.", "artist(s)_name": "Kendrick Lamar", '
             '"artist_count": 1, "released_year": 2017, "released_month": 3, '
             '"released_day": 30, "in_spotify_playlists": 33206, '
             '"in_spotify_charts": 1, "streams": "1929770265", '
             '"in_apple_playlists": 284, "in_apple_charts": 114, '
             '"in_deezer_playlists": "1,481", "in_deezer_charts": 0, '
             '"in_shazam_charts": "5", "bpm": 150, "key": "C#", "mode": '
             '"Minor", "danceability_%": 91, "valence_%": 42, "energy_%": 60, '
             '"acousticness_%": 0, "instrumentalness_%": 0, "liveness_%": 9, '
   

In [49]:
available_functions = filter_functions(all_functions, function_metadata)

conversation = Conversation()
conversation.add({"role": Role.SYSTEM.value, "content": "Answer briefly."}) \
            .add({"role": Role.USER.value, "content": "What are the top 5 Albums?"})

result = chat(conversation, model=Model.GPT3.value, functions=available_functions)
print(result)

The top 5 albums are:

1. "The Black Saint and the Sinner Lady" by Mingus
   - Release Date: 1963-07-01
   - Genres: Avant-Garde Jazz, Third Stream
   - Average Rating: 4.34
   - Number of Ratings: 21,489
   - Number of Reviews: 369

2. "Wish You Were Here" by Pink Floyd
   - Release Date: 1975-09-12
   - Genres: Progressive Rock, Art Rock
   - Average Rating: 4.30
   - Number of Ratings: 51,246
   - Number of Reviews: 1,006

3. "To Pimp a Butterfly" by Kendrick Lamar
   - Release Date: 2015-03-15
   - Genres: Conscious Hip Hop, West Coast Hip Hop, Jazz Rap
   - Average Rating: 4.30
   - Number of Ratings: 47,821
   - Number of Reviews: 415

4. "A Love Supreme" by John Coltrane
   - Release Date: 1965-02-01
   - Genres: Spiritual Jazz
   - Average Rating: 4.30
   - Number of Ratings: 26,404
   - Number of Reviews: 441

5. "In the Court of the Crimson King" by King Crimson
   - Release Date: 1969-10-10
   - Genres: Progressive Rock, Art Rock
   - Average Rating: 4.30
   - Number of Rati

In [2]:
# pprint(conversation.messages_as_dicts)

#### Use Web Server API

In [20]:
# change description of metadata
with open('functions.json', 'r') as file:
    function_metadata = json.load(file)

In [5]:
conversation = Conversation()
conversation.add({"role": Role.SYSTEM.value, "content": "Answer briefly."}) \
            .add({"role": Role.USER.value, "content": "Can you please provide the top five albums based on average rating?"})

result = chat_web(conversation, model=Model.GPT3.value,
              function_metadata=function_metadata)
print(result)

APIError: Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} {'Date': 'Wed, 08 Nov 2023 14:10:53 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT', 'Server': 'cloudflare', 'CF-RAY': '822e66beed601d84-FRA', 'alt-svc': 'h3=":443"; ma=86400'}

In [10]:
pprint(conversation.messages_as_dicts)

[{'content': 'Answer briefly.', 'role': 'system'},
 {'content': 'How many top albums did Nirvana release?', 'role': 'user'},
 {'content': None,
  'function_call': <OpenAIObject at 0x204e18e9df0> JSON: {
  "name": "albums_by_artist",
  "arguments": "{\n  \"artist_name\": \"Nirvana\"\n}"
},
  'role': 'assistant'},
 {'content': '[{"release_name": "Nevermind", "artist_name": "Nirvana", '
             '"release_date": "1991-09-24", "primary_genres": "Grunge, '
             'Alternative Rock", "secondary_genres": "Punk Rock", '
             '"descriptors": "energetic, rebellious, angry, malevocals, '
             'apathetic, sarcastic, alienation, passionate, anxious, '
             'self-hatred", "avg_rating": 3.93, "rating_count": 45503, '
             '"review_count": 947}, {"release_name": "In utero", '
             '"artist_name": "Nirvana", "release_date": "1993-09-21", '
             '"primary_genres": "Grunge", "secondary_genres": "Noise Rock, '
             'Post-Hardcore", "descrip